In [33]:
import cv2
from model import *
from detect import *

In [34]:
model = load_model('dataset/yolov3.cfg', "dataset/yolov3.weights")

In [35]:
img = cv2.imread('dataset/test/volley-51-_png.rf.73e74eadfa77ff53d769279f73561040.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [36]:
boxes = detect_image(model, img)
print(boxes)

[[324.91785    471.4717     380.76727    593.00256      0.99186856
    0.        ]
 [600.41376    463.2613     635.6562     627.0618       0.97510374
    0.        ]
 [205.09041    354.5581     273.91425    497.59485      0.97374356
    0.        ]
 [499.14615    224.63321    525.9882     330.41174      0.9711335
    0.        ]
 [297.84186    241.36615    325.54428    363.3          0.9355484
    0.        ]
 [552.2327     296.06696    584.99255    402.6693       0.8999846
    0.        ]
 [264.9513     205.7782     286.98605    279.979        0.8945052
    0.        ]
 [386.87424    219.32555    407.0504     302.19836      0.86883736
    0.        ]
 [461.89725    320.64447    491.1655     433.82928      0.8406424
    0.        ]
 [368.5119     267.90958    397.88       386.61127      0.72747284
    0.        ]
 [214.90784    287.40277    235.5029     365.02078      0.7268624
    0.        ]
 [ 75.71053    307.0703     104.13161    373.20328      0.65650624
    0.        ]]


In [17]:
len(boxes)

12

In [31]:
options = dict()
options['gpus'] = '0,1,2,3'
options['num_workers'] = '10'
path = 'dataset/yolov3.cfg'
options.pop(0)

KeyError: 0

In [32]:
smth = {'gpus': '0,1,2,3', 'num_workers': '10'}